In [ ]:
print("Hello")

Hello


In [ ]:
!pip install -U accelerate bitsandbytes datasets peft transformers
!pip install huggingface_hub
!pip install -q -U trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.w

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from google.colab import drive
import pickle

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, BitsAndBytesConfig, set_seed
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from trl import setup_chat_format, DPOTrainer
from transformers.trainer_utils import get_last_checkpoint
from accelerate import Accelerator
import logging
import transformers
import datasets
import sys
import os

logger = logging.getLogger(__name__)


# Set seed for reproducibility
set_seed(42)

In [ ]:
# Implemented based on the HuggingFace repository: https://github.com/huggingface/alignment-handbook/blob/main/scripts/run_sft.py

In [ ]:
output_dir = "/content/drive/MyDrive/AI/SycophanticDataGen-DPO/"
model_id="mistralai/Mistral-7B-Instruct-v0.2"

In [ ]:
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)
log_level = "INFO"
logger.setLevel(log_level)
datasets.utils.logging.set_verbosity(log_level)
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()

In [ ]:
last_checkpoint = None
if os.path.isdir(output_dir):
    last_checkpoint = get_last_checkpoint(output_dir)

if last_checkpoint is not None:
    logger.info(f"Checkpoint detected, resuming training at {last_checkpoint=}.")


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    revision="main",
    trust_remote_code=False,
    add_eos_token=True,
    use_fast=True
    )
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = 'left'

# Set reasonable default for models without max length
if tokenizer.model_max_length > 100_000:
    tokenizer.model_max_length = 2048

# TODO, I'm not sure, whether I should use chat template for the other dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

[INFO|tokenization_utils_base.py:2108] 2024-05-29 20:58:31,575 >> loading file tokenizer.model from cache at /root/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.2/snapshots/41b61a33a2483885c981aa79e0df6b32407ed873/tokenizer.model
[INFO|tokenization_utils_base.py:2108] 2024-05-29 20:58:31,576 >> loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.2/snapshots/41b61a33a2483885c981aa79e0df6b32407ed873/tokenizer.json
[INFO|tokenization_utils_base.py:2108] 2024-05-29 20:58:31,577 >> loading file added_tokens.json from cache at None
[INFO|tokenization_utils_base.py:2108] 2024-05-29 20:58:31,578 >> loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.2/snapshots/41b61a33a2483885c981aa79e0df6b32407ed873/special_tokens_map.json
[INFO|tokenization_utils_base.py:2108] 2024-05-29 20:58:31,579 >> loading file tokenizer_config.json from cache at /root/.c

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_storage="uint8",
)



model_kwargs = dict(
    revision="main",
    trust_remote_code=False,
    torch_dtype=torch.bfloat16,
    use_cache=False,
    quantization_config=quantization_config,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    revision="main",
    trust_remote_code=False,
    # use_flash_attention_2=True,
    torch_dtype=torch.bfloat16,
    use_cache=False,
    device_map="auto",
    # device_map={"": Accelerator().local_process_index},
    quantization_config=quantization_config,
)

# I've skipped
# model, tokenizer = setup_chat_format(model, tokenizer)
# model_kwargs = None

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

[INFO|configuration_utils.py:733] 2024-05-29 20:58:32,460 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.2/snapshots/41b61a33a2483885c981aa79e0df6b32407ed873/config.json
[INFO|configuration_utils.py:796] 2024-05-29 20:58:32,466 >> Model config MistralConfig {
  "_name_or_path": "mistralai/Mistral-7B-Instruct-v0.2",
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-05,
  "rope_theta": 1000000.0,
  "sliding_window": null,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.41.1",
  "use_cache": false,
  "vocab_size": 32000
}



model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

[INFO|modeling_utils.py:3474] 2024-05-29 20:58:33,360 >> loading weights file model.safetensors from cache at /root/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.2/snapshots/41b61a33a2483885c981aa79e0df6b32407ed873/model.safetensors.index.json


model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

...

[INFO|modeling_utils.py:1519] 2024-05-29 20:59:20,126 >> Instantiating MistralForCausalLM model under default dtype torch.bfloat16.
[INFO|configuration_utils.py:962] 2024-05-29 20:59:20,129 >> Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2,
  "use_cache": false
}

[INFO|quantizer_bnb_4bit.py:105] 2024-05-29 20:59:21,666 >> target_dtype {target_dtype} is replaced by `CustomDtype.INT4` for 4-bit BnB quantization


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

[INFO|modeling_utils.py:4280] 2024-05-29 20:59:27,412 >> All model checkpoint weights were used when initializing MistralForCausalLM.

[INFO|modeling_utils.py:4288] 2024-05-29 20:59:27,414 >> All the weights of MistralForCausalLM were initialized from the model checkpoint at mistralai/Mistral-7B-Instruct-v0.2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use MistralForCausalLM for predictions without further training.


generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

[INFO|configuration_utils.py:917] 2024-05-29 20:59:28,143 >> loading configuration file generation_config.json from cache at /root/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.2/snapshots/41b61a33a2483885c981aa79e0df6b32407ed873/generation_config.json
[INFO|configuration_utils.py:962] 2024-05-29 20:59:28,144 >> Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2
}



In [ ]:
model = prepare_model_for_kbit_training(model)
model.config.pad_token_id = tokenizer.pad_token_id

peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.05,
        r=16,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules= ['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"]
)

In [ ]:
training_args = TrainingArguments(
    optim="rmsprop",
    output_dir=output_dir,
    save_steps=25,
    logging_steps=25,
    eval_steps=50,
    learning_rate=5.0e-7,
    do_eval=True
)


[INFO|training_args.py:2052] 2024-05-29 20:59:45,389 >> PyTorch: setting up devices
[INFO|training_args.py:1727] 2024-05-29 20:59:45,410 >> The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
NLP_SPLITTER = 'Which label would you assign for the following input'

def clean_from_user_opinion(text):
  return "Human: " + NLP_SPLITTER + text.split(NLP_SPLITTER)[1].strip()

def wrap_human_as_instruct(text):
  split_assistant = text.split('\n\nAssistant:')

  return "[INST]" + split_assistant[0].strip() + "[/INST]\n\nAssistant:" + split_assistant[1].strip()

In [ ]:
with open('/content/drive/MyDrive/AI/nlp_cai_revisions_3_1000.pickle', 'rb') as file:
  dataset_1 = pickle.load(file)

with open('/content/drive/MyDrive/AI/nlp_cai_revisions_3_1_275.pickle', 'rb') as file:
  dataset_2 = pickle.load(file)

dataset = dataset_1 | dataset_2

with open('/content/drive/MyDrive/AI/nlp_cai_revisions_3_2_100.pickle', 'rb') as file:
  eval_dataset_sycophantic = pickle.load(file)

In [ ]:
from datasets import Dataset
import pandas as pd

dataset_dicts = [{"key": key, "correct_letter": correct_letter, "opinion_letter": opinion_letter, "answer": answer, "revision_answer": revision_answer}
                 for (key, (correct_letter, opinion_letter, answer, critique_statement, critique_answer, revision_statement, revision_answer)) in dataset.items()]

for element_dict in dataset_dicts:
  sycophantic_input = element_dict["key"]
  # This part is present in the revision answer already
  element_dict["prompt"] = sycophantic_input
  element_dict["chosen"] = "<s>" + wrap_human_as_instruct(sycophantic_input) + element_dict["revision_answer"].replace("I believe the best answer is", "")
  element_dict["rejected"] = "<s>" + wrap_human_as_instruct(sycophantic_input) + element_dict["answer"]

sycophantic_dataset_for_training = Dataset.from_pandas(pd.DataFrame(dataset_dicts))

In [ ]:
dataset_dicts = [{"key": key, "correct_letter": correct_letter, "opinion_letter": opinion_letter, "answer": answer, "revision_answer": revision_answer}
                 for (key, (correct_letter, opinion_letter, answer, critique_statement, critique_answer, revision_statement, revision_answer)) in eval_dataset_sycophantic.items()]

for element_dict in dataset_dicts:
  sycophantic_input = element_dict["key"]
  # This part is present in the revision answer already
  element_dict["prompt"] = sycophantic_input
  element_dict["chosen"] = "<s>" + wrap_human_as_instruct(sycophantic_input) + element_dict["revision_answer"].replace("I believe the best answer is", "")
  element_dict["rejected"] = "<s>" + wrap_human_as_instruct(sycophantic_input) + element_dict["answer"]

sycophantic_dataset_for_eval = Dataset.from_pandas(pd.DataFrame(dataset_dicts))

In [ ]:
from datasets import load_dataset, concatenate_datasets

dataset_instruct = load_dataset("HuggingFaceH4/ultrafeedback_binarized", split="train_prefs[:2%]")
dataset_instruct = dataset_instruct.shuffle(seed=42).select(range(1000))

dataset_instruct_eval = load_dataset("HuggingFaceH4/ultrafeedback_binarized", split="test_prefs[:20%]")
dataset_instruct_eval = dataset_instruct_eval.shuffle(seed=42).select(range(100))

https://huggingface.co/datasets/HuggingFaceH4/ultrafeedback_binarized/resolve/daee4cffe8bea93b1cd7874b01c5a6a4ae9dcaf2/README.md not found in cache or force_download set to True, downloading to /root/.cache/huggingface/datasets/downloads/7e9a6504fba200de555df5a08d348af6dc0e29396f7221a27b59d8952ad096a7.005a41f17f69620105b3681adac719c611101cd3fb151102124339cd0a6c50af.incomplete
INFO:datasets.utils.file_utils:https://huggingface.co/datasets/HuggingFaceH4/ultrafeedback_binarized/resolve/daee4cffe8bea93b1cd7874b01c5a6a4ae9dcaf2/README.md not found in cache or force_download set to True, downloading to /root/.cache/huggingface/datasets/downloads/7e9a6504fba200de555df5a08d348af6dc0e29396f7221a27b59d8952ad096a7.005a41f17f69620105b3681adac719c611101cd3fb151102124339cd0a6c50af.incomplete


storing https://huggingface.co/datasets/HuggingFaceH4/ultrafeedback_binarized/resolve/daee4cffe8bea93b1cd7874b01c5a6a4ae9dcaf2/README.md in cache at /root/.cache/huggingface/datasets/downloads/7e9a6504fba200de555df5a08d348af6dc0e29396f7221a27b59d8952ad096a7.005a41f17f69620105b3681adac719c611101cd3fb151102124339cd0a6c50af
INFO:datasets.utils.file_utils:storing https://huggingface.co/datasets/HuggingFaceH4/ultrafeedback_binarized/resolve/daee4cffe8bea93b1cd7874b01c5a6a4ae9dcaf2/README.md in cache at /root/.cache/huggingface/datasets/downloads/7e9a6504fba200de555df5a08d348af6dc0e29396f7221a27b59d8952ad096a7.005a41f17f69620105b3681adac719c611101cd3fb151102124339cd0a6c50af
creating metadata file for /root/.cache/huggingface/datasets/downloads/7e9a6504fba200de555df5a08d348af6dc0e29396f7221a27b59d8952ad096a7.005a41f17f69620105b3681adac719c611101cd3fb151102124339cd0a6c50af
INFO:datasets.utils.file_utils:creating metadata file for /root/.cache/huggingface/datasets/downloads/7e9a6504fba200de555d

storing hf://datasets/HuggingFaceH4/ultrafeedback_binarized@daee4cffe8bea93b1cd7874b01c5a6a4ae9dcaf2/data/train_prefs-00000-of-00001.parquet in cache at /root/.cache/huggingface/datasets/downloads/0952e2dd078f219f841c4a5afeedc03ef2cd9ee3b50f1886c60eb2daa3886457
INFO:datasets.utils.file_utils:storing hf://datasets/HuggingFaceH4/ultrafeedback_binarized@daee4cffe8bea93b1cd7874b01c5a6a4ae9dcaf2/data/train_prefs-00000-of-00001.parquet in cache at /root/.cache/huggingface/datasets/downloads/0952e2dd078f219f841c4a5afeedc03ef2cd9ee3b50f1886c60eb2daa3886457
creating metadata file for /root/.cache/huggingface/datasets/downloads/0952e2dd078f219f841c4a5afeedc03ef2cd9ee3b50f1886c60eb2daa3886457
INFO:datasets.utils.file_utils:creating metadata file for /root/.cache/huggingface/datasets/downloads/0952e2dd078f219f841c4a5afeedc03ef2cd9ee3b50f1886c60eb2daa3886457
hf://datasets/HuggingFaceH4/ultrafeedback_binarized@daee4cffe8bea93b1cd7874b01c5a6a4ae9dcaf2/data/train_sft-00000-of-00001.parquet not found i

storing hf://datasets/HuggingFaceH4/ultrafeedback_binarized@daee4cffe8bea93b1cd7874b01c5a6a4ae9dcaf2/data/train_sft-00000-of-00001.parquet in cache at /root/.cache/huggingface/datasets/downloads/0b27041dee67fee2c38750d959023954678acc260596762831b5cc8869bb6685
INFO:datasets.utils.file_utils:storing hf://datasets/HuggingFaceH4/ultrafeedback_binarized@daee4cffe8bea93b1cd7874b01c5a6a4ae9dcaf2/data/train_sft-00000-of-00001.parquet in cache at /root/.cache/huggingface/datasets/downloads/0b27041dee67fee2c38750d959023954678acc260596762831b5cc8869bb6685
creating metadata file for /root/.cache/huggingface/datasets/downloads/0b27041dee67fee2c38750d959023954678acc260596762831b5cc8869bb6685
INFO:datasets.utils.file_utils:creating metadata file for /root/.cache/huggingface/datasets/downloads/0b27041dee67fee2c38750d959023954678acc260596762831b5cc8869bb6685
hf://datasets/HuggingFaceH4/ultrafeedback_binarized@daee4cffe8bea93b1cd7874b01c5a6a4ae9dcaf2/data/test_prefs-00000-of-00001.parquet not found in c

storing hf://datasets/HuggingFaceH4/ultrafeedback_binarized@daee4cffe8bea93b1cd7874b01c5a6a4ae9dcaf2/data/test_prefs-00000-of-00001.parquet in cache at /root/.cache/huggingface/datasets/downloads/a1c7fadf39cb571baee0f3a3b182f96699d63bb58af70d517ab903c5a6fc19e8
INFO:datasets.utils.file_utils:storing hf://datasets/HuggingFaceH4/ultrafeedback_binarized@daee4cffe8bea93b1cd7874b01c5a6a4ae9dcaf2/data/test_prefs-00000-of-00001.parquet in cache at /root/.cache/huggingface/datasets/downloads/a1c7fadf39cb571baee0f3a3b182f96699d63bb58af70d517ab903c5a6fc19e8
creating metadata file for /root/.cache/huggingface/datasets/downloads/a1c7fadf39cb571baee0f3a3b182f96699d63bb58af70d517ab903c5a6fc19e8
INFO:datasets.utils.file_utils:creating metadata file for /root/.cache/huggingface/datasets/downloads/a1c7fadf39cb571baee0f3a3b182f96699d63bb58af70d517ab903c5a6fc19e8
hf://datasets/HuggingFaceH4/ultrafeedback_binarized@daee4cffe8bea93b1cd7874b01c5a6a4ae9dcaf2/data/test_sft-00000-of-00001.parquet not found in c

...

Generating train_gen split
INFO:datasets.builder:Generating train_gen split


Generating train_gen split:   0%|          | 0/61135 [00:00<?, ? examples/s]

Generating test_gen split
INFO:datasets.builder:Generating test_gen split


Generating test_gen split:   0%|          | 0/1000 [00:00<?, ? examples/s]

All the splits matched successfully.
INFO:datasets.utils.info_utils:All the splits matched successfully.
Dataset ultrafeedback_binarized downloaded and prepared to /root/.cache/huggingface/datasets/HuggingFaceH4___ultrafeedback_binarized/default/0.0.0/daee4cffe8bea93b1cd7874b01c5a6a4ae9dcaf2. Subsequent calls will reuse this data.
INFO:datasets.builder:Dataset ultrafeedback_binarized downloaded and prepared to /root/.cache/huggingface/datasets/HuggingFaceH4___ultrafeedback_binarized/default/0.0.0/daee4cffe8bea93b1cd7874b01c5a6a4ae9dcaf2. Subsequent calls will reuse this data.
Caching indices mapping at /root/.cache/huggingface/datasets/HuggingFaceH4___ultrafeedback_binarized/default/0.0.0/daee4cffe8bea93b1cd7874b01c5a6a4ae9dcaf2/cache-50c354c12d8a3b93.arrow
INFO:datasets.arrow_dataset:Caching indices mapping at /root/.cache/huggingface/datasets/HuggingFaceH4___ultrafeedback_binarized/default/0.0.0/daee4cffe8bea93b1cd7874b01c5a6a4ae9dcaf2/cache-50c354c12d8a3b93.arrow
Overwrite dataset i

In [ ]:
def apply_chat_template(
    example,
    tokenizer,
    auto_insert_empty_system_msg: bool = True,
):
  example["prompt"] = example["prompt"]

  example["chosen"] = tokenizer.apply_chat_template(
      example["chosen"],
      tokenize=False
  )

  example["rejected"] = tokenizer.apply_chat_template(
      example["rejected"],
      tokenize=False
  )

  return example


In [ ]:
dataset_instruct_prepared = dataset_instruct.map(apply_chat_template,
        fn_kwargs={
            "tokenizer": tokenizer
        },
        desc="Applying chat template",
)

dataset_instruct_eval_prepared = dataset_instruct_eval.map(apply_chat_template,
        fn_kwargs={
            "tokenizer": tokenizer
        },
        desc="Applying chat template",
)

Applying chat template:   0%|          | 0/1000 [00:00<?, ? examples/s]

Caching processed dataset at /root/.cache/huggingface/datasets/HuggingFaceH4___ultrafeedback_binarized/default/0.0.0/daee4cffe8bea93b1cd7874b01c5a6a4ae9dcaf2/cache-4ef69ffd16ff3a9a.arrow
INFO:datasets.arrow_dataset:Caching processed dataset at /root/.cache/huggingface/datasets/HuggingFaceH4___ultrafeedback_binarized/default/0.0.0/daee4cffe8bea93b1cd7874b01c5a6a4ae9dcaf2/cache-4ef69ffd16ff3a9a.arrow


Applying chat template:   0%|          | 0/100 [00:00<?, ? examples/s]

Caching processed dataset at /root/.cache/huggingface/datasets/HuggingFaceH4___ultrafeedback_binarized/default/0.0.0/daee4cffe8bea93b1cd7874b01c5a6a4ae9dcaf2/cache-a01f29f3e115f014.arrow
INFO:datasets.arrow_dataset:Caching processed dataset at /root/.cache/huggingface/datasets/HuggingFaceH4___ultrafeedback_binarized/default/0.0.0/daee4cffe8bea93b1cd7874b01c5a6a4ae9dcaf2/cache-a01f29f3e115f014.arrow


In [ ]:
dataset_for_training = concatenate_datasets([dataset_instruct_prepared, sycophantic_dataset_for_training]).shuffle(seed=42)
dataset_for_eval = concatenate_datasets([dataset_instruct_eval_prepared, sycophantic_dataset_for_eval]).shuffle(seed=42).select(range(24))

Some of the datasets have disparate format. Resetting the format of the concatenated dataset.
INFO:datasets.arrow_dataset:Some of the datasets have disparate format. Resetting the format of the concatenated dataset.
Caching indices mapping at /root/.cache/huggingface/datasets/HuggingFaceH4___ultrafeedback_binarized/default/0.0.0/daee4cffe8bea93b1cd7874b01c5a6a4ae9dcaf2/cache-f6da8af3cd8accb6.arrow
INFO:datasets.arrow_dataset:Caching indices mapping at /root/.cache/huggingface/datasets/HuggingFaceH4___ultrafeedback_binarized/default/0.0.0/daee4cffe8bea93b1cd7874b01c5a6a4ae9dcaf2/cache-f6da8af3cd8accb6.arrow
Some of the datasets have disparate format. Resetting the format of the concatenated dataset.
INFO:datasets.arrow_dataset:Some of the datasets have disparate format. Resetting the format of the concatenated dataset.
Caching indices mapping at /root/.cache/huggingface/datasets/HuggingFaceH4___ultrafeedback_binarized/default/0.0.0/daee4cffe8bea93b1cd7874b01c5a6a4ae9dcaf2/cache-c82f5074

In [23]:
trainer = DPOTrainer(
    model,
    None,
    model_init_kwargs=None,
    ref_model_init_kwargs=None,
    args=training_args,
    beta=0.1,
    # TODO
    train_dataset=dataset_for_training,
    eval_dataset=dataset_for_eval,
    tokenizer=tokenizer,
    max_prompt_length=512,
    max_length=1024,
    peft_config=peft_config,
    loss_type="sigmoid",
)

trainer.train()

# So, it looks like, I will process ~400 examples during this training, it will take roughly ~2:20
# I could project that SFT for the CAI case will likely require ~1500 examples (+1500 Instruct Examples), processing of these examples will likely take ~19 hours
# Once it is done, my next plan will be to generate another 1500 examples and set up RLHF on them. I could take the new data or stay on the same data as well.


/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:332: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Caching processed dataset at /root/.cache/huggingface/datasets/HuggingFaceH4___ultrafeedback_binarized/default/0.0.0/daee4cffe8bea93b1cd7874b01c5a6a4ae9dcaf2/cache-6e4995278b3a6cae.arrow
INFO:datasets.arrow_dataset:Caching processed dataset at /root/.cache/huggingface/datasets/HuggingFaceH4___ultrafeedback_binarized/default/0.0.0/daee4cffe8bea93b1cd7874b01c5a6a4ae9dcaf2/cache-6e4995278b3a6cae.arrow
[WARNING|tokenization_utils_base.py:3921] 2024-05-29 21:00:10,657 >> Token indices sequence length is longer than the specified maximum sequence length for this model (3287 > 2048). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/24 [00:00<?, ? examples/s]

Caching processed dataset at /root/.cache/huggingface/datasets/HuggingFaceH4___ultrafeedback_binarized/default/0.0.0/daee4cffe8bea93b1cd7874b01c5a6a4ae9dcaf2/cache-da8b83ee46c66d0c.arrow
INFO:datasets.arrow_dataset:Caching processed dataset at /root/.cache/huggingface/datasets/HuggingFaceH4___ultrafeedback_binarized/default/0.0.0/daee4cffe8bea93b1cd7874b01c5a6a4ae9dcaf2/cache-da8b83ee46c66d0c.arrow
[INFO|trainer.py:2078] 2024-05-29 21:00:19,476 >> ***** Running training *****
[INFO|trainer.py:2079] 2024-05-29 21:00:19,477 >>   Num examples = 2,000
[INFO|trainer.py:2080] 2024-05-29 21:00:19,479 >>   Num Epochs = 3
[INFO|trainer.py:2081] 2024-05-29 21:00:19,481 >>   Instantaneous batch size per device = 8
[INFO|trainer.py:2084] 2024-05-29 21:00:19,482 >>   Total train batch size (w. parallel, distributed & accumulation) = 8
[INFO|trainer.py:2085] 2024-05-29 21:00:19,483 >>   Gradient Accumulation steps = 1
[INFO|trainer.py:2086] 2024-05-29 21:00:19,483 >>   Total optimization steps = 750

Step,Training Loss
25,0.669700
50,0.635200
75,0.627100
100,0.619500
125,0.602800
150,0.584800
175,0.567300
200,0.575100
225,0.538900
250,0.534000


[INFO|trainer.py:3410] 2024-05-29 21:20:16,934 >> Saving model checkpoint to /content/drive/MyDrive/AI/SycophanticDataGen-DPO/checkpoint-25
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
[INFO|configuration_utils.py:733] 2024-05-29 21:20:17,548 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.2/snapshots/41b61a33a2483885c981aa79e0df6b32407ed873/config.json
[INFO|configuration_utils.py:796] 2024-05-29 21:20:17,550 >> Model config MistralConfig {
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_po

Step,Training Loss
25,0.669700
50,0.635200
75,0.627100
100,0.619500
125,0.602800
150,0.584800
175,0.567300
200,0.575100
225,0.538900
250,0.534000


[INFO|trainer.py:3410] 2024-05-30 06:55:41,915 >> Saving model checkpoint to /content/drive/MyDrive/AI/SycophanticDataGen-DPO/checkpoint-750
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
[INFO|configuration_utils.py:733] 2024-05-30 06:55:42,531 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.2/snapshots/41b61a33a2483885c981aa79e0df6b32407ed873/config.json
[INFO|configuration_utils.py:796] 2024-05-30 06:55:42,534 >> Model config MistralConfig {
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_p

TrainOutput(global_step=750, training_loss=0.5028367335001628, metrics={'train_runtime': 35724.1782, 'train_samples_per_second': 0.168, 'train_steps_per_second': 0.021, 'total_flos': 0.0, 'train_loss': 0.5028367335001628, 'epoch': 3.0})

In [ ]:
from transformers import GenerationConfig
from peft import PeftModel
model.config.use_cache = True
model = PeftModel.from_pretrained(model, output_dir+'checkpoint-250/')
def generate(instruction):
    prompt = "### Human: "+instruction+"### Assistant: "
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    generation_output = model.generate(
            input_ids=input_ids,
            generation_config=GenerationConfig(pad_token_id=tokenizer.pad_token_id, temperature=1.0, top_p=1.0, top_k=50, num_beams=1),
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=256
    )
    for seq in generation_output.sequences:
        output = tokenizer.decode(seq)
        print(output.split("### Assistant: ")[1].strip())
generate("Tell me about gravitation.")

In [ ]:
# from google.colab import runtime
# runtime.unassign()
